<a href="https://colab.research.google.com/github/LucasFreitas98/Gamer-SnakeXFrog---Python/blob/main/modelo_regressao_linear_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=19e114c660d0e943844aded5517c1bc35e324340e3b2d481a1db33e9d95b92c4
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql import *
import pyspark
import csv

In [3]:
spark=SparkSession.builder.master("local[*]").appName("kaggle.com").getOrCreate()

In [5]:
iris = spark.read.option("header",True).csv("Iris.csv", sep=",")

In [6]:
iris.show(10)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
+---+-------------+------------+-------------+------------+-----

In [9]:
formula = RFormula(formula="Species ~ . ", featuresCol="features", labelCol="label", handleInvalid="skip")

In [10]:
iris_trans = formula.fit(iris).transform(iris).select("features","label")

In [11]:
iris_trans.show(4)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(268,[0,150,192,2...|  0.0|
|(268,[62,157,183,...|  0.0|
|(268,[73,173,185,...|  0.0|
|(268,[84,164,186,...|  0.0|
+--------------------+-----+
only showing top 4 rows



In [12]:
iris_trans = formula.fit(iris).transform(iris).select("features","label")

In [ ]:
iris_trans.show(150)

In [16]:
irisTreino, irisTeste = iris_trans.randomSplit([.7,.3])

In [17]:
nb = NaiveBayes(labelCol="label",featuresCol="features")

In [18]:
modelo = nb.fit(irisTreino)

In [19]:
previsao = modelo.transform(irisTeste)

In [20]:
previsao.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(268,[2,152,184,2...|  1.0|[-30.869038624010...|[0.00352702466761...|       1.0|
|(268,[6,151,188,2...|  2.0|[-30.869038624010...|[0.00826884641608...|       2.0|
|(268,[10,180,188,...|  2.0|[-30.869038624010...|[0.07551642316390...|       1.0|
|(268,[12,158,199,...|  0.0|[-23.835532139722...|[0.99870077623923...|       0.0|
|(268,[17,152,190,...|  2.0|[-30.869038624010...|[0.03453014269790...|       2.0|
|(268,[18,155,184,...|  2.0|[-30.869038624010...|[0.00720894055503...|       2.0|
|(268,[22,168,194,...|  2.0|[-25.473140929123...|[0.01828496796117...|       2.0|
|(268,[24,160,196,...|  2.0|[-31.562185804570...|[0.01265177469519...|       1.0|
|(268,[33,156,184,...|  2.0|[-31.562185804570...|[0.00133525810899...|       2.0|
|(268,[44,160,18

In [21]:
avaliar = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="label",metricName="accuracy")

In [22]:
resultado = avaliar.evaluate(previsao)

In [23]:
print (resultado)

0.9302325581395349
